In [9]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import datetime as dt
import plotly.express as px

from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)
import scripts.functions as fn

from scripts.indicators import resample
import pandas_ta 

from binance.client import Client

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [10]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [11]:
environment = 'klines'
#environment = 'binance'

if environment == 'klines':

	klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
	with open(klines_file, 'rb') as file:
		df = pickle.load(file)
		#df.set_index('datetime',inplace=True)

	df = resample(df,(4*60))
else:
	#Conexion con Binance sin API-KEY
	client = Client()

	start = '2024-09-01'
	end = '2024-12-13'
	symbol = 'BTCUSDT'
	interval = '1h'
	real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	utc_start = real_start+' 00:00:00'
	utc_end = real_end+' 23:59:59'
	df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)


In [12]:
rsi_length = 14
rsi_signal_length = 20
df['RSI'] = pandas_ta.rsi(df['close'],length=rsi_length)
df['RSI_MAX'] = df['RSI'].rolling(window=rsi_signal_length).max()
df['RSI_SIGNAL'] = np.where(df['RSI']>=df['RSI_MAX'],df['close'],None)


In [15]:
df_to_show = df.copy() #[0:20]
df_to_show['price'] = df_to_show['close']
df_to_show.drop(columns={'volume'},inplace=True)

indicators = [
     {'col': 'RSI_SIGNAL','color': 'green','row': 1, 'mode':'markers',},
     #{'col': 'long_fbe_0','name': 'long_fbe_0','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'long_fbe_1','name': 'long_fbe_1','color': 'white','row': 1, 'mode':'markers',},
     #{'col': 'fb_2','name': 'fb_2','color': 'red','row': 2, 'mode':'markers',},
     #{'col': 'stop_loss','name': 'stop_loss','color': 'gray','row': 1, 'mode':'markers',},
     #{'col': 'take_profit','name': 'take_profit','color': 'gray','row': 1, 'mode':'markers',},
        ]

indicators_out = [
     #{'col': 'trend','name': 'trend','color': 'yellow','row': 1, 'mode':'markers',},
     #{'col': 'maxs_up','name': 'maxs_up','color': 'green','row': 2, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       #{'df':df_to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       #{'df':df_to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()
